In [1]:
import warnings

# Set a global filter to ignore the specific DeprecationWarning from jupyter_client
# This is just needed cause something was happening in my Colab notebook where a deprecation message was popping up
warnings.filterwarnings(
    "ignore",
    message="datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).",
    category=DeprecationWarning,
    module="jupyter_client"
)

In [2]:
!pip install --quiet --upgrade \
  google-colab==1.0.0 \
  requests==2.32.4 \
  langchain==0.3.27 \
  langchain-core==0.3.79 \
  huggingface_hub \
  langchain_huggingface \
  langchain-community \
  duckduckgo_search \
  ddgs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.8/503.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.0.1 which is incompatible.


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

question = input("What is your question? ")

template = "{question}"

prompt = PromptTemplate(template=template, input_variables=["question"])

model_id = "Qwen/Qwen2.5-1.5B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    max_new_tokens=256,
)

llm = HuggingFacePipeline(pipeline=pipe)
llm_chain = prompt | llm

response = llm_chain.invoke({"question": question})

print(response)

What is your question? Who is the current President of the United States of America?


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Who is the current President of the United States of America? The current President of the United States of America is Joe Biden. He took office on January 20, 2021. Prior to his election as president in November 2020, he had served as Vice President since taking office following Donald Trump's defeat in the 2016 presidential election. Before that, he was a Senator from Delaware for many years and previously served as Attorney General of Delaware. His term will end on January 20, 2025.


In [4]:
# We run this seperately just in case - probably isnt needed
pip install transformers -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.1
    Uninstalling huggingface-hub-1.0.1:
      Successfully uninstalled huggingface-hub-1.0.1


In [2]:
# Using Langchain to search the webs

from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.tools import BraveSearch
from langchain_core.runnables import RunnableLambda

question = input("What is your question? ")

# Replace with your actual Brave API key
brave_api = "BRAVE_API_KEY_HERE"

search = BraveSearch.from_api_key(api_key=brave_api, search_kwargs={"count": 5})
def fetch_results(q):
  return {"question":q, "search_results": search.run(q)}

# Template that is fed into the LLM model
template = """Use the web search results to answer the question.
Cite sources inline by title if helpful.

Question: {question}

Web search results:
{search_results}

Answer:"""

prompt = PromptTemplate(template=template)

model_id = "Qwen/Qwen2.5-1.5B-Instruct"
pipe = pipeline("text-generation", model=model_id, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

llm_chain = RunnableLambda(fetch_results) | prompt | llm
print(llm_chain.invoke(question))

What is your question? Who is the current President of the United States of America?


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Use the web search results to answer the question.
Cite sources inline by title if helpful.

Question: Who is the current President of the United States of America?

Web search results:
[{"title": "Presidents, vice presidents, and first ladies | USAGov", "link": "https://www.usa.gov/presidents", "snippet": "U.S. head of state ... The 47th and current president of the United States is <strong>Donald John Trump</strong>."}, {"title": "President of the United States - Wikipedia", "link": "https://en.wikipedia.org/wiki/President_of_the_United_States", "snippet": "Ronald Reagan, who had been an ... the Cold War, the United States became the world&#x27;s undisputed leading power. Bill Clinton, George W. Bush, and <strong>Barack Obama</strong> each served two terms as president...."}, {"title": "The White House", "link": "https://www.whitehouse.gov/", "snippet": "President <strong>Donald J. Trump</strong> and Vice President JD Vance are committed to lowering costs for all Americans, securing 